In [175]:
# Imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import color
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [176]:
# Variables
train_num = 50000
valid_num = 20000
test_num = 25000
width = 32
height = 32
num_channels = 1
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [179]:
# Load small training and testing data and labels into memory
train_mat = scipy.io.loadmat('svhn_small/train_32x32.mat')
test_mat = scipy.io.loadmat('svhn_small/test_32x32.mat')
train_X = np.rollaxis(train_mat['X'][:,:,:,:train_num], 3)
train_y = train_mat['y'][:train_num,:]
valid_X = np.rollaxis(train_mat['X'][:,:,:,train_num:train_num + valid_num], 3)
valid_y = train_mat['y'][train_num:train_num + valid_num,:]
test_X = np.rollaxis(test_mat['X'][:,:,:,:test_num], 3)
test_y = test_mat['y'][:test_num,:]

print 'Train data shape: ' + str(train_X.shape)
print 'Train labels shape: ' +  str(train_y.shape)
print 'Valid data shape: ' + str(valid_X.shape)
print 'Valid labels shape: ' +  str(valid_y.shape)
print 'Test data shape: ' + str(test_X.shape)
print 'Test labels shape: ' + str(test_y.shape)

Train data shape: (50000, 32, 32, 3)
Train labels shape: (50000, 1)
Valid data shape: (20000, 32, 32, 3)
Valid labels shape: (20000, 1)
Test data shape: (25000, 32, 32, 3)
Test labels shape: (25000, 1)


In [166]:
# Convert images to grayscale
train_X_gray = np.ndarray((width, height, train_num))
valid_X_gray = np.ndarray((width, height, valid_num))
test_X_gray = np.ndarray((width, height, test_num))

for i in xrange(train_num):
    train_X_gray[:,:,i] = color.rgb2gray(train_X[:,:,:,i])

for i in xrange(valid_num):
    valid_X_gray[:,:,i] = color.rgb2gray(valid_X[:,:,:,i])
    
for i in xrange(test_num):
    test_X_gray[:,:,i] = color.rgb2gray(test_X[:,:,:,i])
    
print 'Train data grayscale shape: ' + str(train_X_gray.shape)
print 'Valid data grayscale shape: ' + str(valid_X_gray.shape)
print 'Test data grayscale shape: ' + str(test_X_gray.shape)

Train data grayscale shape: (32, 32, 50000)
Valid data grayscale shape: (32, 32, 20000)
Test data grayscale shape: (32, 32, 25000)


In [58]:
# # Lets look at a couple images
# %matplotlib inline
# for i in range(10):
#     plt.figure()
#     plt.imshow(test_X_gray[:,:,i])
#     print test_y[i]

In [59]:
# # Run some simple tests via sklearn
# train_X_flat = np.reshape(train_X_gray, (train_num, width * height))
# test_X_flat = np.reshape(test_X_gray, (test_num, width * height))

# clf= SVC()
# clf.fit(train_X_flat[:1000], train_y[:1000])
# print clf.score(test_X_flat[:1000], test_y[:1000])

In [167]:
# Randomize data
def randomize(data, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_data = data[:,:,permutation]
    shuffled_labels = labels[permutation,:]
    return shuffled_data, shuffled_labels
train_X_gray, train_y = randomize(train_X_gray, train_y)
valid_X_gray, valid_y = randomize(valid_X_gray, valid_y)
test_X_gray, test_y = randomize(test_X_gray, test_y)

In [168]:
train_lens = []
valid_lens = []
test_lens = []

for i in xrange(1, 11):
    print 'Train ' + str(i) + ': ' + str(len([x[0] for x in train_y.tolist() if x[0] == i]))
    
for i in xrange(1, 11):
    print 'Valid ' + str(i) + ': ' + str(len([x[0] for x in valid_y.tolist() if x[0] == i]))
    
for i in xrange(1, 11):
    print 'Test ' + str(i) + ': ' + str(len([x[0] for x in test_y.tolist() if x[0] == i]))

Train 1: 9440
Train 2: 7227
Train 3: 5850
Train 4: 5108
Train 5: 4641
Train 6: 3903
Train 7: 3830
Train 8: 3419
Train 9: 3186
Train 10: 3396
Valid 1: 3828
Valid 2: 2910
Valid 3: 2257
Valid 4: 2028
Valid 5: 1908
Valid 6: 1553
Valid 7: 1516
Valid 8: 1396
Valid 9: 1272
Valid 10: 1332
Test 1: 4902
Test 2: 3989
Test 3: 2775
Test 4: 2423
Test 5: 2287
Test 6: 1897
Test 7: 1928
Test 8: 1606
Test 9: 1523
Test 10: 1670


In [169]:
# Reshape data and labels for TensorFlow
train_X_gray = train_X_gray.reshape(width, height, num_channels, train_num).astype(np.float32)
train_y = (labels == train_y[:]).astype(np.float32)
valid_X_gray = valid_X_gray.reshape(width, height, num_channels, valid_num).astype(np.float32)
valid_y = (labels == valid_y[:]).astype(np.float32)
test_X_gray = test_X_gray.reshape(width, height, num_channels, test_num).astype(np.float32)
test_y = (labels == test_y[:]).astype(np.float32)

print 'Train data shape: ' + str(train_X_gray.shape)
print 'Train labels shape: ' +  str(train_y.shape)
print 'Valid data shape: ' + str(valid_X_gray.shape)
print 'Valid labels shape: ' +  str(valid_y.shape)
print 'Test data shape: ' + str(test_X_gray.shape)
print 'Test labels shape: ' + str(test_y.shape)

Train data shape: (32, 32, 1, 50000)
Train labels shape: (50000, 10)
Valid data shape: (32, 32, 1, 20000)
Valid labels shape: (20000, 10)
Test data shape: (32, 32, 1, 25000)
Test labels shape: (25000, 10)


In [170]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [172]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_X = tf.placeholder(tf.float32, shape=(batch_size, width, height, num_channels))
    tf_train_y = tf.placeholder(tf.float32, shape=(batch_size, len(labels)))
    tf_valid_X = tf.constant(valid_X_gray)
    tf_test_X = tf.constant(test_X_gray)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([width // 4 * height // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_X)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_y))
      
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_X))
    test_prediction = tf.nn.softmax(model(tf_test_X))

ValueError: Dimensions Dimension(20000) and Dimension(1) are not compatible